<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [22]:
import pandas as pd
import numpy as np
import scipy

In [2]:
df = pd.read_table('water.txt')
df.describe()

,mortality,hardness
count,61.000000,61.000000
mean,1524.147541,47.180328
std,187.668754,38.093966
min,1096.000000,5.000000
25%,1379.000000,14.000000
50%,1555.000000,39.000000
75%,1668.000000,75.000000
max,1987.000000,138.000000


In [6]:
df.corr(method='pearson')

,mortality,hardness
mortality,1.000000,-0.654849
hardness,-0.654849,1.000000


In [5]:
df.corr(method='spearman')

,mortality,hardness
mortality,1.000000,-0.631665
hardness,-0.631665,1.000000


In [9]:
df[df.location == 'South'].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


In [11]:
df[df.location == 'North'].corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


In [16]:
men = np.concatenate((np.ones(239), np.zeros(515)))
women = np.concatenate((np.ones(203), np.zeros(718)))

In [21]:
x0 = 515
x1 = 239
y0 = 718
y1 = 203

a = x0 + y0
b = x0 + y1
c = x1 + y0
d = x1 + y1

# matthew's correlation 
(a*d - b*c) / (np.sqrt((a+b) * (a+c) * (b+d) * (c+d)))

-0.05397936447957069

In [26]:
scipy.stats.chi2_contingency([[x0, x1], [y0, y1]])

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[555.03402985, 198.96597015],
        [677.96597015, 243.03402985]]))

In [30]:
# интервал разности долей

def proportions_diff_confint_ind(pos1, n1, pos2, n2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = pos1 / n1
    p2 = pos2 / n2
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

proportions_diff_confint_ind(x1, x0+x1, y1, y0+y1)

(0.053905233215813156, 0.13922183141523897)

In [31]:
# z-статистика для проверки гипотезы 

def proportions_diff_z_stat_ind(pos1, n1, pos2, n2):

    
    p1 = pos1 / n1
    p2 = pos2 / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

z_stat = proportions_diff_z_stat_ind(x1, x0+x1, y1, y0+y1)
z_stat

4.46111444482329

In [32]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)
    
pvalue = proportions_diff_z_test(z_stat, alternative='greater')

print("Pvalue:", pvalue)

Pvalue: 4.0767265447883005e-06


In [42]:
chi2_value, p_value = scipy.stats.chi2_contingency(np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]]))[:2]
chi2_value, p_value

(293.68311039689746, 2.4964299580093467e-62)

In [43]:
def cramers_stat(confusion_matrix, chi2_value):

    n = confusion_matrix.sum()
    return np.sqrt(chi2_value / (n*(min(confusion_matrix.shape)-1)))

In [46]:
cramers_stat(np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]]), chi2_value)

0.2412013934500338